# Basic Exploratory Data Analysis

На любом этапе прикидывать реалистичность данных.  
На любом этапе очищать данные от обнаруженных неточностей, несоответствий.  

Готовые "библиотеки" для анализа  
!pip install pandas_profiling --upgrade  
from pandas_profiling import ProfileReport  
profile = ProfileReport(df)  
profile

-----
- Предварительный просмотр данных
  - df.sample(5)
- Переименование полей
  - df.columns = df.columns.str.lower().str.replace(' ',  '\_')
- Общее количество записей и типы столбцов
  - df.shape, 
  - df.info()
- Проверить пропуски
  - df.isnull().sum(), 
  - df.isnull().mean()
- Проверить дубликаты записей
  - df[df.duplicated()].shape[0], 
  - df.duplicated().mean(), 
  - df.drop_duplicates(subset=None, keep=’first’, inplace=False)
- Удалить ненужные колонки
  - из df.columns скопировать названия во внутрь df[[]] и убирать колонки, переходя на новую строку и "комментируя" названия колонок. Чтобы можно было переиграть если что
- Уникальность значений в каждом поле 
  - количество df.nunique(),
  - и сами значения np.sort(df.field.unique()) на случай опечаток
- Преобразование типов данных
  - df.describe(include='object'),
  - время df.pickup = pd.to_datetime(df.pickup),
  - категориальные типы данных (при небольшом количестве значений?)  
    for col in ('color', 'payment', 'pickup_borough', 'dropoff_borough'):
    df[col] = df[col].astype('category')
- На основе столбика с датой-временем создать столбик с датой (месяцем, годом...?)
  - df['day'] = df.datetime_col.dt.date
- ВременнЫе границы, пропущенные периоды
  - df.dates.min(), df.dates.max(),
  - df.dates.dt.to_period("M").nunique() - количеством имеющихся периодов,
  - барплот "периоды - количество строк" sns.countplot(df.dates.dt.to_period("M"))

-----
##### Одномерный анализ
- Посмотреть временные ряды числовых данных (средние?) (по дням, месяцам и годам?). Тенденции, периодичность.
- Гистограмма распределения числовых данных. Посмотреть выбросы.
- Гистограмма распределения числовых данных, в разрезе по различным категориям (пол, тип оплаты...). 
  Если категорий две, то скрипки, если больше, то полупрозрачные гистограммы (или kde)?
- Распределение количества данных по категориальным данным sns.histplot(data=df, x='city_category'); или plt.pie(df.gender.value_counts(), labels=['M', 'F'])

-----
##### Двумерный анализ  
- Функции распределения цели по пользователям/товарам... purchase_by_user = df.groupby('user_id')['purchase'].sum(); sns.ecdfplot(purchase_by_user.values);  
- Гистограммы распределения числовых данных - парное совместное распределение.  
- Диаграммы Параметры категориальные - цель (цель может быть "разной"? Сумма продаж на категорию, средняя сумма продаж на покупателя из категории, средний чек, среднее количество покупок на покупателя в категории...).  
- Двумерные диаграммы. Скаттерплоты + линии регрессии, барплоты "категориальные - числовые". На скаттерплоте категории цветом вывести. Удобнее такое смотреть в BI-системе?  
- Сводные таблицы "Категория - Категория - Числовые данные (цель?)". FasetGrid?
- Pairplot для числовых данных sns.pairplot(df)  
- Корреляционные матрицы df.corr().style.background_gradient(cmap='crest'). С версии 1.5.0 можно не только для числовых столбиков  

-----
- Удалить выбросы? заполнить недостающие значения? Или это уже для ML?  
```python
q_1, q_3 = np.nanpercentile(df.A, [25, 75])
up_thresh = q_3 + 1.5 * diff_3_1
low_thresh = q_1 - 1.5 * diff_3_1
df.A[(df.A > low_thresh) * (df.A < up_thresh)]
```  

-----
- Сопоставить данные, например, в разных таблицах на непротиворечивость: айдишники несущестующих в справочнике пользователей,
дебиторка в 100 раз превышающая отгрузку

-----
- Присоединить новые данные (погода, курсы валют, индексы деловой активности)

-----
Самые важные признаки:
```python
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X = df[['Возраст', 'Длительность', 'Кампания', 'День', 'Предыдущий контакт', 'Индекс потребительских цен', 'Европейская межбанковская ставка', 'Количество сотрудников в компании']]
y = df.iloc[:, -1]
bestfeatures = SelectKBest(score_func = chi2, k = 'all')
fit = bestfeatures.fit(X, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns, dfscores], axis = 1)
featureScores.columns = ['Specs', 'Score']  
print(featureScores.nlargest(10, 'Score')) 
```